In [0]:
import pandas as pd
import numpy as np
import lightgbm as GBM
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


In [79]:
'''
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
'''

'\nfrom google.colab import drive\ndrive.mount("/content/drive", force_remount=True)\n'

In [0]:
# %cd /content/drive/My\ Drive/Kaggle

In [81]:
Income_train_Set = pd.read_csv("income-train.csv")
Income_Pred_Set = pd.read_csv("income-test.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#dedicated for plots
#Income_train_Set.plot.scatter(x = 'Instance', y = 'Income')

In [0]:
_new_column = {'Year of Record':'Year',
               'Housing Situation':'Housing',
               'Crime Level in the City of Employement':'Crime',
               'Work Experience in Current Job [years]':'Experience',
               'Satisfation with employer':'Satisfaction',
               'Size of City':'City',
               'University Degree':'Degree',
               'Wears Glasses':'Glasses',
               'Hair Color':'Hair',
               'Body Height [cm]':'Height',
               'Yearly Income in addition to Salary (e.g. Rental Income)':'Add_Income',
               'Total Yearly Income [EUR]': 'Income'}
Income_train_Set = Income_train_Set.rename(_new_column, axis = 1)
Income_Pred_Set = Income_Pred_Set.rename(_new_column, axis = 1)

In [0]:
#replace 'EUR' in Add_Income
Income_train_Set['Add_Income'] = Income_train_Set['Add_Income'].map(lambda x : x.replace('EUR', ''))
Income_Pred_Set['Add_Income'] = Income_Pred_Set['Add_Income'].map(lambda x : x.replace('EUR', ''))

In [85]:
Income_Pred_Set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369438 entries, 0 to 369437
Data columns (total 17 columns):
Instance        369438 non-null int64
Year            367944 non-null float64
Housing         369438 non-null object
Crime           369438 non-null int64
Experience      369438 non-null object
Satisfaction    356006 non-null object
Gender          343130 non-null object
Age             369438 non-null int64
Country         369437 non-null object
City            369438 non-null int64
Profession      368397 non-null object
Degree          340700 non-null object
Glasses         369438 non-null int64
Hair            344787 non-null object
Height          369438 non-null int64
Add_Income      369438 non-null object
Income          0 non-null float64
dtypes: float64(2), int64(6), object(9)
memory usage: 47.9+ MB


## IQR
#### Removing data led to poor performance 

In [0]:
def subset_by_iqr(dataframe, column, whisker_width=1.1):
    """Remove outliers from a dataframe by column, including optional 
       whiskers, removing outliers.
    """
    # Calculate Q1, Q2 and IQR
    q1 = dataframe[column].quantile(0.25)                 
    q3 = dataframe[column].quantile(0.75)
    iqr = q3 - q1
    # Apply filter with respect to IQR, including optional whiskers
    filter = (dataframe[column] >= q1 - whisker_width*iqr) & (dataframe[column] <= q3 + whisker_width*iqr)
    return dataframe.loc[filter]                  

In [0]:
#Income_train_Set = subset_by_iqr(Income_train_Set, 'Income', whisker_width=4.5)

In [0]:
#Income_train_Set.plot.scatter(x = 'Instance', y = 'Income')

In [89]:
 Income_train_Set.shape ## Important for de-concat

(1048574, 17)

In [90]:
concat_data = pd.concat([Income_train_Set,Income_Pred_Set],ignore_index=True)

concat_data

,Instance,Year,Housing,Crime,Experience,Satisfaction,Gender,Age,Country,City,Profession,Degree,Glasses,Hair,Height,Add_Income,Income
0,1,1940.0,0,33,17,Unhappy,other,45,Afghanistan,25179,group head,No,1,Black,182,0,6182.05
1,2,1940.0,0,25,4.9,Unhappy,female,17,Afghanistan,2278204,heavy vehicle and mobile equipment service tec...,No,0,Blond,172,0,6819.69
2,3,1940.0,0,34,21,Unhappy,female,48,Afghanistan,822134,sorter,Bachelor,0,Blond,144,0,8663.53
3,4,1940.0,0,70,18,Average,female,42,Albania,59477,quality control senior engineer,No,1,Brown,152,0,2400.64
4,5,1940.0,0,51,8,Happy,other,15,Albania,23494,logistician,Master,1,Black,180,0,2816.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418007,369434,1992.0,Small House,177,27,Average,female,68,Ireland,713916,sales representative,No,1,Black,170,0,NaN
1418008,369435,2014.0,Medium Apartment,35,13,Somewhat Happy,NaN,31,Eritrea,2263260,labor standards investigator,Master,0,Blond,153,0,NaN
1418009,369436,1957.0,nA,83,13,Average,female,24,Turkmenistan,643201,senior general counsel,NaN,0,Black,166,68980.82,NaN
1418010,369437,2003.0,Large Apartment,101,17,Happy,male,40,Cuba,444850,personnel coordinator,Master,0,Blond,180,0,NaN


#### fillnan with mean / most_frequent did not receive great improvement

In [91]:
'''
fillna_dict = {'Satisfaction':'Average',
               'Gender':'male',
               'Country': 'Honduras',
               'Profession': 'payment analyst',
               'Degree': 'Bachelor',
               'Hair': 'Black'
               }
'''

"\nfillna_dict = {'Satisfaction':'Average',\n               'Gender':'male',\n               'Country': 'Honduras',\n               'Profession': 'payment analyst',\n               'Degree': 'Bachelor',\n               'Hair': 'Black'\n               }\n"

In [92]:
'''
for key in fillna_dict.keys():
    Concat_Set[key] = Concat_Set[key].fillna(fillna_dict[key])
'''

'\nfor key in fillna_dict.keys():\n    Concat_Set[key] = Concat_Set[key].fillna(fillna_dict[key])\n'

In [93]:
concat_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1418012 entries, 0 to 1418011
Data columns (total 17 columns):
Instance        1418012 non-null int64
Year            1412505 non-null float64
Housing         1418012 non-null object
Crime           1418012 non-null int64
Experience      1418012 non-null object
Satisfaction    1366493 non-null object
Gender          1317577 non-null object
Age             1418012 non-null int64
Country         1418011 non-null object
City            1418012 non-null int64
Profession      1414118 non-null object
Degree          1308674 non-null object
Glasses         1418012 non-null int64
Hair            1323150 non-null object
Height          1418012 non-null int64
Add_Income      1418012 non-null object
Income          1048574 non-null float64
dtypes: float64(2), int64(6), object(9)
memory usage: 183.9+ MB


In [0]:
#convert Object to float; to_numeric doesn't work
concat_data['Add_Income'] = concat_data['Add_Income'].astype(float)
concat_data['Add_Income'] = concat_data['Add_Income'].astype(int)
# downcast = 'float' did not receive improvement


In [95]:
concat_data

,Instance,Year,Housing,Crime,Experience,Satisfaction,Gender,Age,Country,City,Profession,Degree,Glasses,Hair,Height,Add_Income,Income
0,1,1940.0,0,33,17,Unhappy,other,45,Afghanistan,25179,group head,No,1,Black,182,0,6182.05
1,2,1940.0,0,25,4.9,Unhappy,female,17,Afghanistan,2278204,heavy vehicle and mobile equipment service tec...,No,0,Blond,172,0,6819.69
2,3,1940.0,0,34,21,Unhappy,female,48,Afghanistan,822134,sorter,Bachelor,0,Blond,144,0,8663.53
3,4,1940.0,0,70,18,Average,female,42,Albania,59477,quality control senior engineer,No,1,Brown,152,0,2400.64
4,5,1940.0,0,51,8,Happy,other,15,Albania,23494,logistician,Master,1,Black,180,0,2816.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418007,369434,1992.0,Small House,177,27,Average,female,68,Ireland,713916,sales representative,No,1,Black,170,0,NaN
1418008,369435,2014.0,Medium Apartment,35,13,Somewhat Happy,NaN,31,Eritrea,2263260,labor standards investigator,Master,0,Blond,153,0,NaN
1418009,369436,1957.0,nA,83,13,Average,female,24,Turkmenistan,643201,senior general counsel,NaN,0,Black,166,68980,NaN
1418010,369437,2003.0,Large Apartment,101,17,Happy,male,40,Cuba,444850,personnel coordinator,Master,0,Blond,180,0,NaN


#### different method of creating new features are tested,<br/>including:<br/>
1.Separate features into categorical ones and linear one. Combining each count value of catergorical features (make multiplication or division) with linear values. (Received score around 11k, meeting overfitting problem)<br/>
2.Creating new features by counting the number of different individuals in different categorical (finally selected)

In [96]:
'''
def new_feature(dataframe,Categorical_Value, Linear_Value,normalize=True):
    for i,col_value in enumerate(Categorical_Value):
        Col_val = df[col_value].value_counts(dropna=False, normalize=normalize).to_dict() #normalize = true showing different proportino
        New_col = col_value + '_NF' 
        dataframe[New_col] = df[col_value].map(Col_val) 
        dataframe[New_col] = df[New_col].astype('float32')
        for j,con in enumerate(Linear_Value):
            new_col = Categorical_Value +'_'+ Linear_Value
            print('timeblock frequency encoding:', new_col)
            dataframe[new_col] = dataframe[cat].astype(str)+'_'+df[con].astype(str)  
            temp_df = df[new_col]
            fq_encode = temp_df.value_counts(normalize=True).to_dict()
            dataframe[new_col] = dataframe[new_col].map(fq_encode)
            dataframe[new_col] = dataframe[new_col]*dataframe[col_value+'_NF']
    return df

'''

"\ndef new_feature(dataframe,Categorical_Value, Linear_Value,normalize=True):\n    for i,col_value in enumerate(Categorical_Value):\n        Col_val = df[col_value].value_counts(dropna=False, normalize=normalize).to_dict() #normalize = true showing different proportino\n        New_col = col_value + '_NF' \n        dataframe[New_col] = df[col_value].map(Col_val) \n        dataframe[New_col] = df[New_col].astype('float32')\n        for j,con in enumerate(Linear_Value):\n            new_col = Categorical_Value +'_'+ Linear_Value\n            print('timeblock frequency encoding:', new_col)\n            dataframe[new_col] = dataframe[cat].astype(str)+'_'+df[con].astype(str)  \n            temp_df = df[new_col]\n            fq_encode = temp_df.value_counts(normalize=True).to_dict()\n            dataframe[new_col] = dataframe[new_col].map(fq_encode)\n            dataframe[new_col] = dataframe[new_col]*dataframe[col_value+'_NF']\n    return df\n\n"

In [0]:
def create_new_feature(Set,feature): # sort by counts and concat to original dataframe
    feat_count = Set.groupby([feature]).size().reset_index()
    feat_count.columns = [feature,'%s_count'%(feature)]
    # replace original feature column with counts value and sorted value
    Set = Set.merge(feat_count,how='left',on=[feature])
    return Set

In [0]:
Col_list = concat_data.columns.tolist()
except_col = ['Income', 'Instance']
Feature_col = [value for value in concat_data.columns if value not in except_col]

#### binary Encoder did not work well; planned to use target encoder but did not have time to try; Selected LabelEncoder

```python
!pip install category_encoders #INSTALL TARGET ENCODER
```

In [0]:
#Feature_col

In [100]:
for col in Feature_col:
    concat_data = create_new_feature(concat_data,col)

Object_cols = concat_data[Feature_col].dtypes[concat_data[Feature_col].dtypes == 'object'].index.tolist()
for col in Object_cols:
    le = LabelEncoder()
    concat_data[col] = le.fit_transform(concat_data[col].astype(str))
                          
'Loading---100%'

'Loading---100%'

In [0]:
train_set = concat_data[concat_data['Income'].notnull()]
test_set = concat_data[concat_data['Income'].isnull()]      

In [0]:
correlation = train_set.corr()
correlation_inc = correlation['Income']
abs_corr = correlation_inc.abs().sort_values()

In [0]:
drop_corr = abs_corr.loc[abs_corr < 0.0006]
list_of_unrelated = drop_corr.index.tolist()
#count = test.groupby('Age').size().reset_index()


In [0]:
for value in list_of_unrelated:
  train_set = train_set.drop(value, axis = 1)
  test_set = test_set.drop(value, axis = 1)
  

In [0]:
Feature_col = list(set(Feature_col) - set(list_of_unrelated))

In [106]:
train_x,val_x,train_y,val_y = train_test_split(train_set[Feature_col],train_set['Income'],test_size=0.3,random_state=1234)

'Loading---100%'

'Loading---100%'

In [107]:




#parameters tried
parameters = {
              'max_depth': [15, 20, 25, 30, 35],
              'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15],
              'feature_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              'bagging_freq': [2, 4, 5, 6, 8],
              'lambda_l1': [0, 0.1, 0.4, 0.5, 0.6],
              'lambda_l2': [0, 10, 15, 35, 40],
              'cat_smooth': [1, 10, 15, 20, 35],
              'objective' : ['regression', 'quantile', 'mape', 'gamma', 'tweedie']}

Pars = {
          'max_depth': 30,
          'learning_rate': 0.1,
          "boosting": "gbdt",
          "bagging_seed": 11,
          "metric": 'mae',
          "verbosity": -1,
          'objective':'tweedie'
         }
training_set = GBM.Dataset(train_x, label=train_y)
valid_set = GBM.Dataset(val_x, label=val_y)
Mod = GBM.train(Pars, training_set, 10000, valid_sets = [training_set, valid_set], verbose_eval=1000, early_stopping_rounds=500)


Training until validation scores don't improve for 500 rounds.
[1000]	training's l1: 8551.75	valid_1's l1: 8993.71
[2000]	training's l1: 7778.91	valid_1's l1: 8678.32
[3000]	training's l1: 7260.85	valid_1's l1: 8535.9
[4000]	training's l1: 6859.98	valid_1's l1: 8465.71
[5000]	training's l1: 6526.78	valid_1's l1: 8419.05
[6000]	training's l1: 6233.39	valid_1's l1: 8385.56
[7000]	training's l1: 5980.73	valid_1's l1: 8364.9
[8000]	training's l1: 5750.3	valid_1's l1: 8346.82
[9000]	training's l1: 5521.36	valid_1's l1: 8340.04
[10000]	training's l1: 5313.54	valid_1's l1: 8330.42
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 5313.54	valid_1's l1: 8330.42


'Loading---100%'

In [0]:
pre_res = Mod.predict(test_set[Feature_col])

'Loading---100%'

In [ ]:
from sklearn.metrics import mean_absolute_error
comp_res = Mod.predict(val_x)
mae_error = mean_absolute_error(val_y,comp_res)

In [111]:
mae_error

8330.4237621802

In [112]:
Result = pd.DataFrame()
Result['Instance'] = test_set['Instance'].tolist()
Result['Total Yearly Income [EUR]'] = pre_res
Result.to_csv("Submission_N.csv",index=False)

'Loading---100%'

'Loading---100%'

####------ End of Code ------